In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.sample import sample_gen
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from pathlib import Path
import math

DATA_DIR = Path('data')
OUTPUT_DIR = Path('output')
KEJADIAN_FILE = DATA_DIR / 'kejadian.csv'
OUTPUT_RASTER_FILE = OUTPUT_DIR / 'peta_kerawanan_purbalingga.tif'

print("--- Proses Evaluasi Model Kriging Dimulai ---")

try:
    df_crime = pd.read_csv(KEJADIAN_FILE)
    print(f"✓ Data kejadian berhasil dimuat ({len(df_crime)} baris).")
except Exception as e:
    print(f"❌ Gagal memuat data kejadian: {e}")
    raise SystemExit()

try:
    raster = rasterio.open(OUTPUT_RASTER_FILE)
    print(f"✓ Raster hasil interpolasi berhasil dimuat: {OUTPUT_RASTER_FILE.name}")
except Exception as e:
    print(f"❌ Gagal memuat file raster: {e}")
    raise SystemExit()

lons = df_crime['longitude'].values
lats = df_crime['latitude'].values
values_asli = df_crime['jumlah_kejadian'].values

coords = [(lon, lat) for lon, lat in zip(lons, lats)]

interpolated_values = []

for val in raster.sample(coords):
    v = val[0]
    interpolated_values.append(v if not math.isnan(v) else np.nan)

interpolated_values = np.array(interpolated_values, dtype=float)

mask_valid = ~np.isnan(interpolated_values)
values_asli_valid = values_asli[mask_valid]
prediksi_valid = interpolated_values[mask_valid]

print(f"✓ {mask_valid.sum()} titik valid digunakan untuk evaluasi dari total {len(values_asli)} titik.")

mse = mean_squared_error(values_asli_valid, prediksi_valid)
mape = mean_absolute_percentage_error(values_asli_valid, prediksi_valid) * 100 

print("\n--- Hasil Evaluasi ---")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")


hasil_df = pd.DataFrame({
    'longitude': lons[mask_valid],
    'latitude': lats[mask_valid],
    'nilai_asli': values_asli_valid,
    'nilai_prediksi': prediksi_valid
})

hasil_df.to_csv(OUTPUT_DIR / 'hasil_evaluasi.csv', index=False)
print(f"✓ Hasil evaluasi per titik disimpan di '{OUTPUT_DIR / 'hasil_evaluasi.csv'}'")

print("\n--- Evaluasi Selesai ---")


--- Proses Evaluasi Model Kriging Dimulai ---
✓ Data kejadian berhasil dimuat (39 baris).
✓ Raster hasil interpolasi berhasil dimuat: peta_kerawanan_purbalingga.tif
✓ 39 titik valid digunakan untuk evaluasi dari total 39 titik.

--- Hasil Evaluasi ---
Mean Squared Error (MSE): 0.0302
Mean Absolute Percentage Error (MAPE): 4.43%
✓ Hasil evaluasi per titik disimpan di 'output\hasil_evaluasi.csv'

--- Evaluasi Selesai ---
